# Function Approximation
In the previous chapters, each update can only modify value estimate for one state, or one state-action pair. 

Unfortunately, state spaces and action spaces can be very large in some tasks, even infinite. It would be impossible to update all state-action pairs individually.

Function approximation tries to address this problem by approximating these values using parametric function, so updating the parameters of this parametric function can update the value estimaes of lots of states or state-action pairs.

When updating parameters according to experience, value estimates of states of state-action pairs that we have not visited yet can also be updated.

## Basics
FA uses mathematical model to estimate real value. For MC and TD algorithms, target to approximate are values (including optimal ones), such as $v_\pi, q_\pi, v_*, q_*$.

Will consider optimising policy and dynamics in remaining chapters of the book.

### Machine Learning: Parametric Model and Nonparametric Model
Parametric models predefine the functional form of the model.

E.g: Feature Vector $\vec{x}$, target vector $\vec{y}$, with relationship $\vec{y} = \vec{F}(\vec{x})$ where $\vec{F}$ is the mapping of some kind.

We want to model the mapping somehow. A parametric model assumes that the model has some form $\vec{y} = \vec{f}(\vec{x}; \vec{w}) + \vec{n}$.

Linear Model Example:
$$f(\vec{x}; \vec{w}) = \vec{x}^\top\vec{w} = \sum_{j \in J}x_jw_j$$
Simple and easy to understand, but it is limited to being a linear relationship when it is much more likely in practice that it is nonlinear.